In [108]:
import bs4
import requests
from typing import List
import pandas as pd
import sqlite3 as db
import os

# Библиотечные функции для упрощения парсинга данных
def get_house_list_url(offset: int, limit: int) -> str:
    return 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={}&limit={}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'.format(offset, limit)

def get_house_info_url(id: int) -> str:
    return 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{}'.format(id)

def get_number_of_houses() -> int:
    return requests.get(get_house_list_url(0, 1)).json()['data']['total']

def get_house_info(id: int) -> dict:
    return requests.get(get_house_info_url(id)).json()

def get_house_info_ids(offset: int, limit: int) -> List[int]:
    return list(map(lambda x: x['objId'], requests.get(get_house_list_url(offset, limit)).json()['data']['list']))

def percentage(current_number, total_number):
    return (total_number / 100) * current_number

def parse_house_info(list_request_size: int = 100, percent_of_all_houses: int = 100, initial_offset: int = 0) -> List[dict]:
    total_number_of_houses = get_number_of_houses()
    number_of_houses_to_parse = int((total_number_of_houses/100.0) * percent_of_all_houses)
    print("houses will be parsed: {}".format(number_of_houses_to_parse))
    offset = initial_offset
    house_infos: List[dict] = []
    while offset < number_of_houses_to_parse:
        print("downloading {}%".format(percentage(offset, number_of_houses_to_parse)))
        house_info_ids = get_house_info_ids(offset, list_request_size)
        for id in house_info_ids:
            house_infos.append(get_house_info(id))
        offset += list_request_size
    print("downloading 100%")
    return house_infos

# На вход принимает массив возвращаемый функцией parse_house_info
def download_history_images(house_infos):
    for house_index, house_info in enumerate(house_infos):
        print(f"Обрабатываем дом: {house_index+1}/{len(house_infos)}")
        house_id = house_info['data']['id']
        r = requests.get("https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3-%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BA/%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82/{}".format(house_id))
        soup = bs4.BeautifulSoup(r.text, 'html.parser')
        images = soup.findAll('img')
        images = list(map(lambda i: i.get("src"), images))
        images = list(filter(lambda i: 'filename=' in i, images))
        image_dir = "./images/{}".format(house_id)
        if len(images) > 0 and not os.path.exists(image_dir):
            os.makedirs(image_dir)
        for index, image_url in enumerate(images):
            print(f"Скачиваем изображение: {index+1}/{len(images)}")
            image_bytes = requests.get(image_url).content
            if image_bytes:
                image_name = image_url.split('filename=')[-1]
                with open(f"{image_dir}/{index+1}-{image_name}", "wb") as file:
                    file.write(image_bytes)



In [109]:
house_infos = parse_house_info(10, 1, 0)

houses will be parsed: 100
downloading 0.0%
downloading 10.0%
downloading 20.0%
downloading 30.0%
downloading 40.0%
downloading 50.0%
downloading 60.0%
downloading 70.0%
downloading 80.0%
downloading 90.0%
downloading 100%


In [110]:
print(len(house_infos))
print(house_infos[0])

100
{'data': {'id': 13391, 'pdId': 28101, 'developer': {'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП', 'devShortNm': '2МЕН ГРУПП', 'devFullCleanNm': '2МЕН ГРУПП', 'problObjCnt': 0, 'buildObjCnt': 5, 'comissObjCnt': 0, 'regRegionDesc': 'Тюменская область', 'devPhoneNum': '+7(345)279-18-88', 'devSite': '2mengroup.ru', 'devEmail': 'info@2mengroup.ru', 'devInn': '7701651356', 'devOgrn': '1067746424899', 'devKpp': '720301001', 'devLegalAddr': 'Тюменская область, город Тюмень, улица Комсомольская дом 75/5', 'devFactAddr': 'Тюменская область, город Тюмень, улица Комсомольская 75/5', 'lastRpdId': 585074, 'fundGuarantyFlg': 1, 'devOrgRegRegionCd': 72, 'devEmplMainFullNm': 'Киселев Сергей Михайлович', 'developerGroupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ', 'orgForm': {'id': 1, 'fullForm': 'Акционерное общество', 'shortForm': 'АО'}, 'companyGroupId': 699421001, 'objGuarantyEscrowFlg': 0, 'govFundFlg': 0}, 'region': 72, 'address': 'г Тюмень, ул Профсоюзная', 'floorMin': 3, 'floorMax': 12, 'objElemLivingCn

In [111]:
df = pd.json_normalize(house_infos)

In [112]:
df.to_excel("nash_dom.xlsx")
df.to_pickle("nash_dom.pkl")

In [113]:
df.drop(df.columns[[50, 86, 99]], axis=1, inplace=True)
con = db.connect('nash_dom.db')
df.to_sql(name='nash_dom', if_exists='replace', con=con)

100

In [115]:
download_history_images(house_infos[0:3])

Обрабатываем дом: 1/2
Скачиваем изображение: 1/29
Скачиваем изображение: 2/29
Скачиваем изображение: 3/29
Скачиваем изображение: 4/29
Скачиваем изображение: 5/29
Скачиваем изображение: 6/29
Скачиваем изображение: 7/29
Скачиваем изображение: 8/29
Скачиваем изображение: 9/29
Скачиваем изображение: 10/29
Скачиваем изображение: 11/29
Скачиваем изображение: 12/29
Скачиваем изображение: 13/29
Скачиваем изображение: 14/29
Скачиваем изображение: 15/29
Скачиваем изображение: 16/29
Скачиваем изображение: 17/29
Скачиваем изображение: 18/29
Скачиваем изображение: 19/29
Скачиваем изображение: 20/29
Скачиваем изображение: 21/29
Скачиваем изображение: 22/29
Скачиваем изображение: 23/29
Скачиваем изображение: 24/29
Скачиваем изображение: 25/29
Скачиваем изображение: 26/29
Скачиваем изображение: 27/29
Скачиваем изображение: 28/29
Скачиваем изображение: 29/29
Обрабатываем дом: 2/2
Скачиваем изображение: 1/28
Скачиваем изображение: 2/28
Скачиваем изображение: 3/28
Скачиваем изображение: 4/28
Скачиваем из